In [ ]:
import copy
import itertools
import functools
import collections
import operator
import sys
import re
import numpy as np
import math

In [ ]:
from helpers.functions import *

Configuration

In [ ]:
DIR = "data/2018/"
load_day = functools.partial(load, DIR)

# Problems

## Day 6

http://adventofcode.com/2018/day/6

In [ ]:
content = load_day(6)

points = list(map(lambda line: list(map(int, line.split(','))), content))
# points = [[1, 1], [1, 6], [8, 3], [3, 4], [5, 5], [8, 9]]  # Example

min_x = min(map(lambda p: p[0], points))
min_y = min(map(lambda p: p[1], points))
max_x = max(map(lambda p: p[0], points))
max_y = max(map(lambda p: p[1], points))

nb_points = len(points)

# points = list(map(lambda p: [p[0] - min_x, p[1] - min_y], points))
# max_x -= min_x
# max_y -= min_y
# min_x = min_y = 0

__Part 1__

In [ ]:
def distance(from_point, to_point):
    return abs(from_point[0] - to_point[0]) + abs(from_point[1] - to_point[1])

In [ ]:
def store_grid():
    grid_lines = []
    for x_idx in range(min_x, max_x + 1):
        line = ""
        for y_idx in range(min_y, max_y + 1):
            if len(attribution[x_idx - min_x][y_idx - min_y][1]) == 1:
                ref = "A" if attribution[x_idx - min_x][y_idx - min_y][0] == 0 else "a"
                line += chr(ord(ref) + attribution[x_idx - min_x][y_idx - min_y][1][0])
            else:
                line += '.'
        grid_lines += [line]
    return grid_lines

def display_grid(lines=None):
    if lines is None:
        lines = store_grid()

    for line in lines:
        print(line)

In [ ]:
# Version 1
attribution = []

for x_idx in range(min_x, max_x + 1):
    row = []
    for y_idx in range(min_y, max_y + 1):
        elem = []
        for p_idx in range(nb_points):
            d = distance(points[p_idx], [x_idx, y_idx])
            if not elem or d < elem[0]:
                elem = [d, [p_idx]]
            elif d == elem[0]:
                elem = [d, elem[1] + [p_idx]]
        row += [elem]
    attribution += [row]

grid_lines = store_grid()
borders = grid_lines[0] + grid_lines[-1] + ''.join(map(lambda x: x[0], grid_lines)) + ''.join(map(lambda x: x[-1], grid_lines))
excluded_candidates = set(x for x in borders.lower())

counter = collections.Counter([y.lower() for x in grid_lines for y in x])

for candidate in excluded_candidates:
    del counter[candidate]

print(f"Answer 1: {counter.most_common(1)}")

# display_grid(lines=grid_lines)

In [ ]:
# Version 2
counter = collections.Counter()
excluded_candidates = set()

for x_idx in range(min_x, max_x + 1):
    for y_idx in range(min_y, max_y + 1):
        scores = []
        for p_idx in range(nb_points):
            scores += [distance(points[p_idx], [x_idx, y_idx])]

        scores = np.array(scores)
        candidates = np.argwhere(scores == scores.min())
        
        if candidates.size == 1:
            counter[candidates[0][0]] += 1

            if x_idx in [min_x, max_x] or y_idx in [min_y, max_y]:
                excluded_candidates.add(candidates[0][0])
        
for cand in excluded_candidates:
    del counter[cand]

print(f"Answer 1: {counter.most_common(1)}")

__Part 2__

In [ ]:
region_size = 0

for x_idx in range(min_x, max_x + 1):
    for y_idx in range(min_y, max_y + 1):
        score = 0
        for p_idx in range(nb_points):
            score += distance(points[p_idx], [x_idx, y_idx])
            
        if score < 10000:
            region_size += 1

print(f"Answer 2: {region_size}")

## Day 7

http://adventofcode.com/2018/day/7

In [ ]:
content = load_day(7)

__Part 1__

__Part 2__

## Day 8

http://adventofcode.com/2019/day/8

In [ ]:
content = load_day(8)

__Part 1__

__Part 2__